In [2]:
import os
import json
import ee
import geemap
import xarray as xr
import wxee
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [2]:
# ee.Authenticate()

In [3]:
ee.Initialize()
wxee.Initialize()

In [10]:
file_path = os.path.join('../data/inputs/san_jose_aoi/resourceshedbb_CostaRica_SanJose.geojson')

with open(file_path) as f:
    json_data = json.load(f)

bbox_geojson = geojson_to_ee(json_data) # need as a feature collection, not bounding box

In [11]:
## set time frame
before_start= '2017-09-25'
before_end='2017-10-05'

after_start='2017-10-05'
after_end='2017-10-15'

In [12]:
# SET SAR PARAMETERS (can be left default)

# Polarization (choose either "VH" or "VV")
polarization = "VH"  # or "VV"

# Pass direction (choose either "DESCENDING" or "ASCENDING")
pass_direction = "DESCENDING"  # or "ASCENDING"

# Difference threshold to be applied on the difference image (after flood - before flood)
# It has been chosen by trial and error. Adjust as needed.
difference_threshold = 1.25

# Relative orbit (optional, if you know the relative orbit for your study area)
# relative_orbit = 79

In [13]:
# Rename the selected geometry feature
aoi = bbox_geojson

# Load and filter Sentinel-1 GRD data by predefined parameters
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
    .filter(ee.Filter.eq('orbitProperties_pass', pass_direction)) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filterBounds(aoi) \
    .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Create a mosaic of selected tiles and clip to the study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply radar speckle reduction by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine the flood result using additional datasets
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))
flooded_mask = difference_binary.where(swater_mask, 0)
flooded = flooded_mask.updateMask(flooded_mask)
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))


In [14]:
m = geemap.Map()
m.add('layer_manager')
m.setCenter(-84.1, 9.9, 8)
# Add the flooded areas layer to the map with a blue palette
m.addLayer(flooded, {'palette': '0000FF'}, 'Flooded areas')
m

Map(center=[9.9, -84.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [27]:
crs = flooded.projection().crs()

flooded = flooded.setDefaultProjection(crs=crs, scale=10)

scale = 100 # 100m resolution


def reduce_resolution(image, reducer, scale):
    return image.reduceResolution(reducer=reducer, maxPixels=10000) \
                .reproject(crs=crs, scale=scale)

flooded_100m = reduce_resolution(flooded, ee.Reducer.mode(), scale)

In [25]:
aoi_geom = aoi.geometry().bounds()

In [26]:
filename = 'test.tif'
geemap.ee_export_image(flooded_100m, filename, scale=60, crs=crs, region=aoi_geom)

Generating URL ...
An error occurred while downloading.
Image.reduceResolution: Bad maxPixels arg (must be less than 65536).
